In [5]:
import requests
import json
import pandas as pd
import time

In [6]:
# Overpass Queries der möglichen Aufenthaltsorte

wohnort_q ="""area[name=Dortmund]->.ar;
( 
   way(area.ar)[building=garage];
   way(area.ar)[building=carport];
   node(area.ar)[building=carport];
   node(area.ar)[building=garage];
);
out center;
"""

arbeitsplatz_q = """
area[name="Dortmund"]->.ar;
(
   way(area.ar)[building=industrial];
   way(area.ar)[building=office];
   way(area.ar)[building=commercial];
   way(area.ar)[office];

   node(area.ar)[building=industrial];
   node(area.ar)[building=office];
   node(area.ar)[building=commercial];
   node(area.ar)[office];
);
out center;
"""

supermarkt_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[building=supermarket];
  way(area.ar)[shop=supermarket];
  node(area.ar)[shop=supermarket];
 );

out center;
"""

dienstleistungen_q = """
[out:json];
area[name="Dortmund"]->.ar;

(
  node(area.ar)[amenity=driving_school];
  node(area.ar)[shop=hairdresser];
  node(area.ar)[shop=car_repair];
  node(area.ar)[shop=laundry];
  node(area.ar)[shop=beauty];
  node(area.ar)[shop=massage];
  
  way(area.ar)[amenity=driving_school];
  way(area.ar)[shop=hairdresser];
  way(area.ar)[shop=car_repair];
  way(area.ar)[shop=laundry];
  way(area.ar)[shop=beauty];
  way(area.ar)[shop=massage];
);

out center;
"""

sonstige_shops_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[building=supermarket];
  way(area.ar)[shop=supermarket];
  way(area.ar)[shop=convenience];
  way(area.ar)[shop=health_food];
  node(area.ar)[shop=supermarket];
  node(area.ar)[shop=convenience];
  node(area.ar)[shop=health_food];
 )->.supermarket;

(
  way(area.ar)[shop];
  node(area.ar)[shop];
 )->.shops;

(
  .shops; - .supermarket;
);

out center;
"""

medizinisch_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[building=supermarket];
  way(area.ar)[shop=supermarket];
  way(area.ar)[shop=convenience];
  way(area.ar)[shop=health_food];
  node(area.ar)[shop=supermarket];
  node(area.ar)[shop=convenience];
  node(area.ar)[shop=health_food];
 )->.supermarket;

(
  way(area.ar)[shop];
  node(area.ar)[shop];
 )->.shops;

(
  .shops; - .supermarket;
);

out center;
"""

# Behörde, Bank, Post
bbp_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[government];
  way(area.ar)[building=government]; 
  way(area.ar)[amenity=bank];
  way(area.ar)[amenity=embassy];
  way(area.ar)[amenity=police];
  way(area.ar)[amenity=post_office];
  
  node(area.ar)[government];
  node(area.ar)[building=government];
  node(area.ar)[amenity=bank];
  node(area.ar)[amenity=embassy];
  node(area.ar)[amenity=police];
  node(area.ar)[amenity=post_office];
);

out center;
"""

# Restaurant, Gaststätte, Kneipe, Disco
restaurants_bars_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[amenity=bar];
  way(area.ar)[amenity=bbq];
  way(area.ar)[amenity=biergarten];
  way(area.ar)[amenity=cafe];
  way(area.ar)[amenity=fast_food];
  way(area.ar)[amenity=food_court];
  way(area.ar)[amenity=ice_cream];
  way(area.ar)[amenity=pub];
  way(area.ar)[amenity=restaurant];
  way(area.ar)[amenity=nightclub];
  way(area.ar)[leisure=dance];
  
  node(area.ar)[amenity=bar];
  node(area.ar)[amenity=bbq];
  node(area.ar)[amenity=biergarten];
  node(area.ar)[amenity=cafe];
  node(area.ar)[amenity=fast_food];
  node(area.ar)[amenity=food_court];
  node(area.ar)[amenity=ice_cream];
  node(area.ar)[amenity=pub];
  node(area.ar)[amenity=restaurant];
  node(area.ar)[amenity=nightclub];
  node(area.ar)[leisure=dance];
);

out center;
"""

religiös_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[building=cathedral];
  way(area.ar)[building=chapel];
  way(area.ar)[building=church];
  way(area.ar)[building=mosque];
  way(area.ar)[building=religious];
  way(area.ar)[building=synagogue];
  way(area.ar)[building=temple];
  way(area.ar)[amenity=grave_yard];
  
  node(area.ar)[building=cathedral];
  node(area.ar)[building=chapel];
  node(area.ar)[building=church];
  node(area.ar)[building=mosque];
  node(area.ar)[building=religious];
  node(area.ar)[building=synagogue];
  node(area.ar)[building=temple];
  node(area.ar)[amenity=grave_yard];
  
  rel(area.ar)[landuse=cemetery];
);

out center;
"""

kulturell_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[amenity="arts_centre"];
  way(area.ar)[amenity=cinema];
  way(area.ar)[amenity=planetarium];
  way(area.ar)[amenity=theatre];
  way(area.ar)[building=museum];
  way(area.ar)[tourism=museum];
  
  node(area.ar)[amenity=arts_centre];
  node(area.ar)[amenity=cinema];
  node(area.ar)[amenity=planetarium];
  node(area.ar)[amenity=theatre];
  node(area.ar)[tourism=museum];
);

out center;
"""

sport_q = """
[out:json];
area[name="Dortmund"]->.ar;
(
  way(area.ar)[building="sports_hall"];
  way(area.ar)[leisure="fitness_centre"];
  way(area.ar)[leisure="fitness_station"];
  way(area.ar)[leisure="golf_course"];
  way(area.ar)[leisure="ice_rink"];
  way(area.ar)[leisure="miniature_golf"];
  way(area.ar)[leisure=track];
  way(area.ar)[leisure="swimming_pool"];
  way(area.ar)[leisure="sports_centre"];
  
  node(area.ar)[building="sports_hall"];
  node(area.ar)[leisure="fitness_centre"];
  node(area.ar)[leisure="fitness_station"];
  node(area.ar)[leisure="golf_course"];
  node(area.ar)[leisure="ice_rink"];
  node(area.ar)[leisure="miniature_golf"];
  node(area.ar)[leisure=track];
  node(area.ar)[leisure="swimming_pool"];
  node(area.ar)[leisure="sports_centre"];
);

out center;
"""

In [7]:
import overpy
import sys

def overpy_query(query):
    while(True):
        try:
            api = overpy.Overpass()
            res = api.query(query)
            break
        # Fängt Exception nicht
        except Exception as e:
            print(e)
            time.sleep(30)
        except:
            print ("Exception:", sys.exc_info()[0])
            print("Erneuter Versuch in 30s.")
            time.sleep(30)
    return res
        

In [8]:
r_wohn = overpy_query(wohnort_q)
r_arbeit = overpy_query(arbeitsplatz_q)
r_super = overpy_query(supermarkt_q)
r_dienst = overpy_query(dienstleistungen_q)
r_shops = overpy_query(sonstige_shops_q)
r_med = overpy_query(medizinisch_q)
r_bbpg = overpy_query(bbp_q)
r_rest = overpy_query(restaurants_bars_q)
r_reli = overpy_query(religiös_q)
r_kult = overpy_query(kulturell_q)
r_sport = overpy_query(sport_q)

In [9]:
def format_res(r):
    r_nodes = [[node.id, float(node.lat), float(node.lon), node.tags] for node in r.nodes]
    r_ways = [[way.id, float(way.center_lat), float(way.center_lon), way.tags] for way in r.ways]
    r_nodes.extend(r_ways)
    return pd.DataFrame(columns=["osm_id", "lat", "lon", "tags"], data=r_nodes)

In [10]:
# Abfragen der Daten über die unterschiedlichen Aufenthaltsorte
wohnort = format_res(r_wohn)
arbeitsplätze = format_res(r_arbeit)
supermärkte = format_res(r_super)
dienstleistungen = format_res(r_dienst)
sonstige_shops = format_res(r_shops)
medizinisch = format_res(r_med)
bbpg = format_res(r_bbpg)
restaurants_bars = format_res(r_rest)
religiös = format_res(r_reli)
kulturell = format_res(r_kult)
sport = format_res(r_sport)

NameError: name 'r_arbeit' is not defined

In [ ]:
# lat und lon Werte von Wegen und Relations in entsprechenden Spalten vermerken
locations = {"Arbeitsplatz":arbeitsplätze, "Supermarkt":supermärkte, "Dienstleistung":dienstleistungen, "Sonstiges Geschäft":sonstige_shops,
             "Medizinisch":medizinisch, "BBPG":bbpg, "Gastronomie":restaurants_bars, "Kirche, Friedhof":religiös,
             "Kulturell":kulturell, "Sportstätten":sport}

In [ ]:
# Berechne Distanz zwischen 2 Punkten über Haversine Formel
def calc_distance(lat_1, lat_2, lon_1, lon_2):
    import math
    # Umrechnung zu rad
    lat_1 = lat_1 * (math.pi / 180)
    lat_2 = lat_2 * (math.pi / 180)
    # Radius der Erde
    r = 6371000
    # delta der Winkel
    d_lon = (lon_2 - lon_1) * (math.pi / 180) 
    d_lat = (lat_2 - lat_1)
    # haversine formula 
    hav = (math.sin(d_lat/2) ** 2) + math.cos(lat_1) * math.cos(lat_2) * (math.sin(d_lon/2) ** 2)
    # Distanz zwischen 2 Punkten mithilfe der Haversine
    d = 2 * r * math.asin(math.sqrt(hav))
    return d
    

In [ ]:
# Test
lat_1 = 42.990967
lon_1 = 71.463767
lat_2 = 43.990967
lon_2 = 72.463767

In [ ]:
print(calc_distance(lat_1, lat_2, lon_1, lon_2))

In [ ]:
# Laden der Parkplatzinformationen
import pickle 
import os

path = os.getcwd()+"\\parkplätze.pickle"
parkplätze = pickle.load(open(path, "rb"))

In [ ]:
def check_access(type_loc, parkplätze, pp_row):
    access = parkplätze.at[pp_row, "access"]
    if access == "yes" or "customers":
        return True
    if access == "private":
        if type_loc == "Arbeitsplatz":
            return True
        else:
            return False

In [ ]:
# Bestimme die nahgegelegensten Parkplätze
for key, loc in locations.items():
    # Spalte für die nahegelegensten Parkplätze
    loc["pp_nearby"] = pd.Series([], dtype='object')
    
    for i in range(len(loc)):
        dist = []
        lat_1 = loc.at[i, "lat"]
        lon_1 = loc.at[i, "lon"]
        for j in range(len(parkplätze)):
            lat_2 = parkplätze.at[j, "lat"]
            lon_2 = parkplätze.at[j, "lon"]
            # berechne Distanz zwischen Ort und Parkplätzen
            d = calc_distance(lat_1, lat_2, lon_1, lon_2)
            if check_access(key, parkplätze, j):
                dist.append((d, parkplätze.at[j, "osm_id"]))
        dist.sort(key=lambda x:x[0])
        loc.at[i, "pp_nearby"] = dist[0:5]

In [ ]:
locations["Wohnort"] = wohnort

In [ ]:
import pickle
import os

path = os.getcwd()+"\\Aufenthaltsorte.pickle"

#pickle.dump(locations, open(path, "wb"))

In [ ]:
locations["Wohnort"].iloc[1]